

**Assignment - Recommender System Assignment ML**

**Deepak Raghavendra**


**TASk 01 : Train Test Split**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [ ]:
spark = SparkSession.builder.master("App").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
spark

**Importing the Libraries**

In [ ]:
import pandas as pd 
import numpy as np

from pyspark.sql import functions as F
# Import for typecasting columns
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType, ArrayType

In [ ]:
raw_ratings_df = (spark.read.csv("raw_ratings_small.csv",
                                 header=True, 
                                 inferSchema= True))

In [ ]:
raw_recipes_df = spark.read.csv("raw_recipies_small.csv",
                                header=True, 
                                inferSchema=True)

In [ ]:
assert (raw_recipes_df.count(), len(raw_recipes_df.columns)) == (20340, 13)
assert (raw_ratings_df.count(), len(raw_ratings_df.columns)) == (93357, 5)

In [ ]:
#finding the data points in the dataset
num_review_int=raw_ratings_df.count()

In [ ]:
num_review_int

93357

In [ ]:
test_num_reviews_int = round(num_review_int *0.2)

In [ ]:
temp_ratings_df = (raw_ratings_df.sort("review_date", ascending=False)
                                 .limit(test_num_reviews_int)
                  )

In [ ]:
assert temp_ratings_df.count()  == 18671
assert raw_recipes_df.collect()[11][4] <= raw_recipes_df.collect()[10][4]

In [ ]:
temp_ratings_df.collect()[-1][4]

'2011-07-17'

In [ ]:
#showing the data for raw_recipes
raw_recipes_df.show(5)

+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
|                name|   id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|year_of_review|
+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------------+
|beat this  banana...|75452|     70|         15892|2003-11-04|['weeknight', 'ti...|[2669.3, 160.0, 9...|     12|['preheat oven to...|  from ann hodgman's|['sugar', 'unsalt...|            9|          2003|
|      chinese  candy|23933|     15|         35268|2002-03-29|['15-minutes-or-l...|[232.7, 21.0, 77....|      4|['melt butterscot...|a little differen...|['butterscotch ch...|     

In [ ]:
#showing the data for raw_ratings
raw_ratings_df.show(5)

+-------+---------+------+--------------------+-----------+
|user_id|recipe_id|rating|              review|review_date|
+-------+---------+------+--------------------+-----------+
| 483827|   306785|     5|Being a  huge fan...| 2008-07-15|
|   6258|    20930|     5|Jan  what an inte...| 2002-07-09|
| 102602|    20930|     5|Jan  we love your...| 2003-10-26|
| 296027|   182985|     4|Very nice fresh s...| 2007-05-19|
|   9580|   208980|     5|I saw this on 30 ...| 2007-02-05|
+-------+---------+------+--------------------+-----------+
only showing top 5 rows



In [ ]:
temp_ratings_df.show(5)

+----------+---------+------+--------------------+-----------+
|   user_id|recipe_id|rating|              review|review_date|
+----------+---------+------+--------------------+-----------+
|2000498330|   314535|     5|Delicious  bright...| 2018-12-19|
|2002296817|   307469|     0|I'm telling you r...| 2018-12-19|
|2001355121|   232811|     3|It was too acidic...| 2018-12-19|
|2002178501|    51803|     5|Omg this was the ...| 2018-12-19|
|2002367593|   209317|     5|I enjoy the sauce...| 2018-12-18|
+----------+---------+------+--------------------+-----------+
only showing top 5 rows



In [ ]:
interaction_level_df = raw_ratings_df.join(
                                           raw_recipes_df,# dataframe 2, 
                                           raw_ratings_df.recipe_id==raw_recipes_df.id,# key to join 
                                           how="left"# how to join    
                                          )

In [ ]:
interaction_level_df.show()

+----------+---------+------+--------------------+-----------+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|   user_id|recipe_id|rating|              review|review_date|                name|   id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|
+----------+---------+------+--------------------+-----------+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|    152552|    29601|     5|Excellent. It was...| 2004-08-20|zippy cold spicy ...|29601|    270|         30367|2002-05-29|['time-to-make', ...|[277.2, 15.0, 13....|     12|['in a heavy 10-i...|shrimp drink in t...|['oil', 'medium s...|           

In [ ]:
# type casting submitted and review_date features into date type and creating other time based
interaction_level_df = (interaction_level_df.withColumn('submitted',F.col("submitted").cast("date"))
                        .withColumn('review_date',F.col("review_date").cast("date")))

In [ ]:
interaction_level_df.show(5)

+-------+---------+------+--------------------+-----------+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|user_id|recipe_id|rating|              review|review_date|                name|   id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|
+-------+---------+------+--------------------+-----------+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
| 152552|    29601|     5|Excellent. It was...| 2004-08-20|zippy cold spicy ...|29601|    270|         30367|2002-05-29|['time-to-make', ...|[277.2, 15.0, 13....|     12|['in a heavy 10-i...|shrimp drink in t...|['oil', 'medium s...|           11|
|  16334

In [ ]:
interaction_level_df.printSchema() #printing the schema of the joined dataframe

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- year_of_review: integer (nullable = true)



In [ ]:
#Now , we will be using the filter command to seperate the datasets
# All interactions which were rated BEFORE '2011-07-17' will be train data. 
train_interaction_level_df=(interaction_level_df.filter(interaction_level_df.review_date<'2011-07-17'))

In [ ]:
train_interaction_level_df.show() #displaying the train dataset

+-------+---------+------+--------------------+-----------+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|user_id|recipe_id|rating|              review|review_date|                name|   id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|
+-------+---------+------+--------------------+-----------+--------------------+-----+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
| 152552|    29601|     5|Excellent. It was...| 2004-08-20|zippy cold spicy ...|29601|    270|         30367|2002-05-29|['time-to-make', ...|[277.2, 15.0, 13....|     12|['in a heavy 10-i...|shrimp drink in t...|['oil', 'medium s...|           11|
|  16334

In [ ]:
train_interaction_level_df.printSchema() #printing the schema of the train dataset

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: date (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)



In [ ]:
#Creating the test dataset
test_interaction_level_df=interaction_level_df.filter(interaction_level_df.review_date>='2011-07-17')

In [ ]:
#showing the test dataset
test_interaction_level_df.show()

+----------+---------+------+--------------------+-----------+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|   user_id|recipe_id|rating|              review|review_date|                name|    id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|
+----------+---------+------+--------------------+-----------+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|1802380878|    35912|     0|I pretty much fol...| 2014-01-19|cheesy bacon   sa...| 35912|     75|         45698|2002-08-02|['bacon', 'weekni...|[4123.9, 503.0, 2...|     13|['preheat oven to...|neat breakfast (o...|['breakfast sausa...|       

In [ ]:
#printing the schema for the test dataset
test_interaction_level_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- year_of_review: integer (nullable = true)



In [ ]:
#creating the data files for modelling
(train_interaction_level_df.coalesce(1)
                           .write.mode('overwrite')
                           .parquet('train_interaction_level_df.parquet'))  # change the file name and file path

In [ ]:
(test_interaction_level_df.coalesce(1)
                                       .write.mode('overwrite')
                                       .parquet('test_interaction_level_df.parquet'))  # change the file name and file pat

**Task 02 : Collaborative filtering model**

 1. Build an ALS-based recommender using the native Spark library.  

2. Create 10 recommendations for each user in the test data. 

**1. Build an ALS based recommender using the native Spark Library**

In [ ]:
try:
  import lenskit 
except:
  %pip install lenskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 KB 11.6 MB/s eta 0:00:00


In [ ]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [ ]:
spark = SparkSession.builder.master("local").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
spark

In [ ]:
#Importing the libraries
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType, ArrayType

In [ ]:
from lenskit import batch, topn, util
from lenskit.algorithms import Recommender, als, item_knn as knn #importing the required functions for ALS and testing

In [ ]:
#Reading the Data
train_ratings_df = spark.read.parquet('train_interaction_level_df.parquet', 
                                      header=True, 
                                      inferSchema=True)

In [ ]:
test_ratings_all_df  = spark.read.parquet('test_interaction_level_df.parquet', 
                                          header=True, 
                                          inferSchema=True)

In [ ]:
raw_recipes_df = spark.read.csv("RAW_recipes_cleaned.csv", 
                                header=True, 
                                inferSchema=True)

In [ ]:
#Counting the number of ratings in the Dataset
numerator = train_ratings_df.select("rating").count()
# Count the number of distinct userIds and distinct recipe_Ids
train_ratings_df.createOrReplaceTempView("RATINGS")
spark.sql("SELECT COUNT(DISTINCT user_id) FROM RATINGS").show()
spark.sql("SELECT COUNT(DISTINCT recipe_id) FROM RATINGS").show()
num_users = 23362 # find the number of unique users in the training data. The output must be an integer
num_recipes = 18721 # find the number of unique recipes in the training data. The output must be an integer
# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_recipes
# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The training dataframe is ", "%.7f" % sparsity + "% empty.")

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                  23362|
+-----------------------+

+-------------------------+
|count(DISTINCT recipe_id)|
+-------------------------+
|                    18721|
+-------------------------+

The training dataframe is  99.9829264% empty.


In [ ]:
def manual_recommendation_check (user_id):
  '''
  Given a user ID form the test dataset, this function will return the names of the recipes recommended to the user. 

  Initialize this function after the all_recs data frame is calculated. 

  Input user_id of a user from the test set as an integer.  
  Prints the names of recipes recommended to this user. 
  Returns nothing. 
  '''
  recs_user = all_recs_als[all_recs_als.user == user_id]
  recs_user_list = list(recs_user.item.values)
  recs_user_list = [x.item() for x in recs_user_list]
  display((raw_recipes_df.filter(F.col('id').isin(recs_user_list))
               .select("name")
               .collect()
               ))

In [ ]:
#creating the ALS model
# Create ALS model
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
als = ALS(maxIter=5, rank=4, regParam=0.01, userCol='user_id', itemCol='recipe_id', ratingCol='rating', coldStartStrategy='drop')

In [ ]:
assert type(als) == pyspark.ml.recommendation.ALS

Training the Model

In [ ]:
model=als.fit(train_ratings_df)

*Predicting the Model*

In [ ]:
#using the model to create the predictions for the test data
test_predictions_unseen=model.transform(test_ratings_all_df)

In [ ]:
assert test_predictions_unseen.select(F.col("prediction"))

In [ ]:
test_predictions_unseen.select(F.col("user_id"),
                               F.col("recipe_id"),
                               F.col("rating"),
                               F.col("prediction")
                              ).show(5)

+-------+---------+------+----------+
|user_id|recipe_id|rating|prediction|
+-------+---------+------+----------+
| 199020|    55265|     5| 2.6867282|
| 369284|    76143|     5| 1.1619632|
| 224235|    89385|     5| -5.064153|
| 385423|    95476|     4|-26.846462|
| 538098|    95476|     5| -5.714027|
+-------+---------+------+----------+
only showing top 5 rows



In [ ]:
# Each user in the test set must have 10 predictions. 
# Use ALS model to get these predictions. 
# You can use the recommendForAllUsers() method. 

recommendations = model.recommendForAllUsers(20).show(10) 

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|  28170|[{393869, 69.7053...|
|  56680|[{258596, 34.0402...|
| 108460|[{107090, 44.0412...|
| 139830|[{143193, 11.2735...|
| 198430|[{294011, 48.7934...|
| 241660|[{282967, 12.5233...|
| 280340|[{226501, 12.9577...|
| 291650|[{106241, 191.472...|
| 343180|[{123766, 70.3455...|
| 409910|[{284599, 44.7136...|
+-------+--------------------+
only showing top 10 rows



In [ ]:
recommendations = model.recommendForAllUsers(10)

In [ ]:
assert len(dict(recommendations.select(F.col("recommendations")).collect()[0][0])) == 10

In [ ]:
# Transfroming the results by exploding the recommendations column

recommendations = (recommendations.select(F.col("user_id"),
                                          F.posexplode(F.col("recommendations")).alias("pos", "item")) 
                                  .select(F.col("user_id"),
                                          F.col("pos"), 
                                          F.col("item.recipe_id").alias("recomended_recipe_id"), 
                                          F.col("item.rating").alias("predicted_rating")))

**Evaluation**

### Task 04 - Model Evaluation

#### 1. RMSE

Add the argument details to the evaluator function below to calculate the RMSE score of the ALS model. 

In [ ]:
# Define evaluator as RMSE and print RMSE value
evaluator_seen = RegressionEvaluator(metricName="rmse", 
                                     labelCol= "rating",  # add the name of the column that has the ratings 
                                     predictionCol= "prediction"# add the name of the column that has the predicted ratings. 
                                     ) 

In [ ]:
predictions=model.transform(test_ratings_all_df)
RMSE = evaluator_seen.evaluate(predictions) # Use the evaluator to find the RMSE on the test set. 
print(RMSE)

8.126878581858024


#### Rank based Metrics

We will use the lenskit library to calculate the ranking-based matrics. The lenskit library is available in pandas only so we need to convert the data frames from PySpark dataframes to Pandas dataframes. 

In [ ]:
all_recs_als = recommendations.toPandas()

In [ ]:
# Rename the columns of to eunsre that they match the columns names as in the cell below. 

column_names = {'user_id':'user','pos':'rank','recomended_recipe_id':'item','predicted_rating':'score'}# create a dictionary with current column names as keys and the intended column names as values. 
all_recs_als = all_recs_als.rename(columns=column_names)

In [ ]:
all_recs_als

,user,rank,item,score
0,28170,0,393869,69.705353
1,28170,1,419104,55.946678
2,28170,2,282967,53.000317
3,28170,3,141570,52.260361
4,28170,4,132675,47.870216
...,...,...,...,...
233615,1904821,5,9410,0.000000
233616,1904821,6,9970,0.000000
233617,1904821,7,10150,0.000000
233618,1904821,8,11440,0.000000


In [ ]:
# Adding a column to make sure the we know these recommendations are from the ALS algorithm. 

all_recs_als["algorithm"] = "ALS"

In [ ]:
# Convert the test dataset to pandas and ensure that it has the same column names as shown in the cell below. 
# Also, note there are only three columns in the rest data. You have to ensure your test data looks identical. 
# test_data = (# add code to select the necessary columns and change the name of the columns. 
#                                 .toPandas())

test_data =test_ratings_all_df.select("user_id", "recipe_id","rating").toPandas()
column_names = {'user_id':'user','recipe_id':'item'}
test_data = test_data.rename(columns=column_names)

In [ ]:
test_data

,user,item,rating
0,1802380878,35912,0
1,2001602879,40335,5
2,2758877,50348,0
3,199020,55265,5
4,369284,76143,5
...,...,...,...
18679,855082,438292,5
18680,1553277,447699,0
18681,2000072578,447699,5
18682,2775141,469503,5


In [ ]:
# code to calculate the necessary metrics 
# the code below uses lenskit library to evaluate the rank metrics

rla = topn.RecListAnalysis()
rla.add_metric(topn.recip_rank)
rla.add_metric(topn.ndcg)
rla.add_metric(topn.dcg)
results = rla.compute(all_recs_als, test_data)
results.head()

nrecs  recip_rank  ndcg  dcg
algorithm user                                
ALS       28170      10         0.0   NaN  NaN
          56680      10         0.0   0.0  0.0
          108460     10         0.0   0.0  0.0
          139830     10         0.0   NaN  NaN
          198430     10         0.0   NaN  NaN

Manual Prediction Checking

1. Why are ndcg and dcg nulls? EG: user 28170

In [ ]:
test_data[test_data.user == 28170]

,user,item,rating


In [ ]:
all_recs_als[all_recs_als.user == 28170]

,user,rank,item,score,algorithm
0,28170,0,393869,69.705353,ALS
1,28170,1,419104,55.946678,ALS
2,28170,2,282967,53.000317,ALS
3,28170,3,141570,52.260361,ALS
4,28170,4,132675,47.870216,ALS
5,28170,5,297381,47.798813,ALS
6,28170,6,268010,47.089630,ALS
7,28170,7,214127,46.031509,ALS
8,28170,8,31379,45.832798,ALS
9,28170,9,62510,43.473637,ALS


User ```28170``` does not appear in the test set. Hence cannot be evaluated.



2. Why are all metrics 0 for specific users?

In [ ]:
test_data[test_data.user == 56680]

,user,item,rating
446,56680,229831,5


In [ ]:
all_recs_als[all_recs_als.user == 56680]

,user,rank,item,score,algorithm
10,56680,0,258596,34.040279,ALS
11,56680,1,395776,33.659737,ALS
12,56680,2,128332,33.632332,ALS
13,56680,3,222852,32.111568,ALS
14,56680,4,402309,31.759281,ALS
15,56680,5,428605,30.356880,ALS
16,56680,6,170626,29.946934,ALS
17,56680,7,370127,29.672890,ALS
18,56680,8,273032,29.200851,ALS
19,56680,9,230863,29.072641,ALS


3. Are any non zero metrics? 

In [ ]:
results[results.ndcg > 0]

nrecs  recip_rank      ndcg       dcg
algorithm user                                          
ALS       64667       10    0.500000  1.000000  3.154649
          280271      10    0.333333  0.048787  2.000000
          199020      10    0.111111  0.058144  1.505150
          1418189     10    0.500000  0.443976  3.154649

In [ ]:
results[results.dcg > 0]

nrecs  recip_rank      ndcg       dcg
algorithm user                                          
ALS       64667       10    0.500000  1.000000  3.154649
          280271      10    0.333333  0.048787  2.000000
          199020      10    0.111111  0.058144  1.505150
          1418189     10    0.500000  0.443976  3.154649

In [ ]:
results[results.recip_rank > 0]

nrecs  recip_rank      ndcg       dcg
algorithm user                                          
ALS       64667       10    0.500000  1.000000  3.154649
          280271      10    0.333333  0.048787  2.000000
          199020      10    0.111111  0.058144  1.505150
          1418189     10    0.500000  0.443976  3.154649

Manually check one prediction

In [ ]:
manual_recommendation_check(653438)

[Row(name='blue cornbread'),
 Row(name='chocolaty and gooey brownies'),
 Row(name='jim s chicken provolone'),
 Row(name='kasha  buckwheat groats  breakfast cereal'),
 Row(name='mackinac island peanut butter fudge'),
 Row(name='mmmmm   muffins   more'),
 Row(name='pork tenderloin cutlets with cranberry maple sauce'),
 Row(name='scallop chowder'),
 Row(name='shrimp veracruz style  camarones a la veracruzana'),
 Row(name='teriyaki beef wraps')]

**Saving the models and predictions**

In [ ]:
all_recs_als.to_csv("recommendation_als.csv", # modify the path 
                    index=False)

In [ ]:
model.save('ALS_model.model') # modify the path